In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [3]:
first_np, second_np, label_np  = data.getData(0)

no_first_np, no_second_np, no_label_np  = data.getData(1)

avg length / 5: 16
avg length / 5: 16


In [4]:
first_np, second_np = utils.process_splitted(first_np, second_np)

no_first_np, no_second_np = utils.process_splitted(no_first_np, no_second_np)

linearized complete!
linearized complete!


In [5]:
no_first_ls, no_second_ls, no_ch2idx, no_max_len = utils.tokenize(no_first_np, no_second_np, '/split/ch2idx_split_yesgudu')
no_first2idx_np, no_second2idx_np = utils.encode(no_first_ls, no_second_ls, no_ch2idx, no_max_len)

first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np, '/split/ch2idx_split_nogudu')
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

done tokenizing both data!
saved ch2idx to file!
encoding comlete!
done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [6]:
# pretrained_word2vec = pm.load_pretrained_model(ch2idx)
# pretrained_word2vec = torch.tensor(pretrained_word2vec)

In [7]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

no_train_first, no_test_first, no_train_second, no_test_second, no_train_labels, no_test_labels = train_test_split(
    no_first2idx_np, no_second2idx_np, no_label_np, test_size = 0.1, random_state = 43
)

no_train_first, no_val_first, no_train_second, no_val_second, no_train_labels, no_val_labels = train_test_split(
    no_train_first, no_train_second, no_train_labels, test_size = 0.1, random_state = 43
)

In [8]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(
                                                                   train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=100)

no_train_dataloader, no_val_dataloader, no_test_dataloader = dl.data_loader(
                                                                   no_train_first,
                                                                   no_train_second,
                                                                   no_val_first,
                                                                   no_val_second,
                                                                   no_test_first,
                                                                   no_test_second,
                                                                   no_train_labels,
                                                                   no_val_labels,
                                                                   no_test_labels,
                                                                   batch_size=100)

In [9]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/final/split')

In [10]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [11]:
# ====================
# set parameters here
# ====================

title = 'split_test_3_'
epochs = 40

vocab_size=len(ch2idx)
embed_dim = 100
hidden_size = 200
num_classes = 2
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.2

learning_rate = 0.001
weight_decay = 1e-4

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=None
freeze_embedding=False,

In [12]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing without pretrained model!!!


/home/arise/anaconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


OCR_rnn(
  (emb): Embedding(121, 100)
  (lstm1): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc1): Linear(in_features=800, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [13]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title+'yes_gudu',
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.011821   | 99.654321  | 0.000096 | 100.00 |  5.05 
   2    |   0.001056   | 99.987654  | 0.000211 | 100.00 |  5.08 
   3    |   0.000842   | 99.991770  | 0.000060 | 100.00 |  5.16 
   4    |   0.002184   | 99.958848  | 0.000040 | 100.00 |  5.07 
   5    |   0.001038   | 99.991770  | 0.000225 | 100.00 |  5.17 
   6    |   0.001364   | 99.983539  | 0.000114 | 100.00 |  5.14 
   7    |   0.001094   | 99.987654  | 0.000074 | 100.00 |  5.16 
   8    |   0.001071   | 99.991770  | 0.000279 | 100.00 |  5.12 
   9    |   0.000998   | 99.991770  | 0.000077 | 100.00 |  5.18 
  10    |   0.001109   | 99.987654  | 0.000065 | 100.00 |  5.19 
  11    |   0.001241   | 99.991770  | 0.000072 | 100.00 |  5.05 
  12    |   0.001108   | 99.987654  | 0.000036 | 100.00 |  5.08 
  13    |   0.001102   | 99.991770  | 0.000087 | 100.

In [14]:
mu.saveModel(title+'yes_gudu', model)

In [15]:
model = mu.getModel(title+'yes_gudu')
print(model)

OCR_rnn(
  (emb): Embedding(121, 100)
  (lstm1): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc1): Linear(in_features=800, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [16]:
loss, acc = tester.test(test_dataloader=no_test_dataloader,
                        device=device,
                        model=model,
                        title=title+'yes_gudu')

test loss:  3.193044726053874
test acc:  49.766666666666666
saved precision and recall results to file!


In [17]:
with open('../result/final_split', 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [18]:
mu.graphModel(train_dataloader, model, writer)

uploaded model graph to tensorboard!


In [19]:
trainer.set_seed(42)

no_model, no_optimizer, no_loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(no_model)

doing without pretrained model!!!
OCR_rnn(
  (emb): Embedding(121, 100)
  (lstm1): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc1): Linear(in_features=800, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


/home/arise/anaconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [20]:

start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title+'no_gudu',
    writer=writer,
    train_dataloader=no_train_dataloader,
    val_dataloader=no_val_dataloader,
    device=device,
    model=no_model,
    optimizer=no_optimizer,
    loss_fn=no_loss_fn
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.033917   | 99.082305  | 0.012442 | 99.56  |  5.02 
   2    |   0.016559   | 99.588477  | 0.009699 | 99.78  |  4.96 
   3    |   0.015139   | 99.621399  | 0.009561 | 99.56  |  5.00 
   4    |   0.012693   | 99.670782  | 0.007856 | 99.78  |  4.97 
   5    |   0.011377   | 99.728395  | 0.009561 | 99.74  |  4.94 
   6    |   0.012337   | 99.691358  | 0.007575 | 99.74  |  4.96 
   7    |   0.012683   | 99.703704  | 0.009154 | 99.74  |  4.96 
   8    |   0.011493   | 99.687243  | 0.009352 | 99.70  |  5.07 
   9    |   0.012844   | 99.666667  | 0.007761 | 99.74  |  5.01 
  10    |   0.011115   | 99.679012  | 0.008377 | 99.78  |  4.95 
  11    |   0.010660   | 99.716049  | 0.007208 | 99.81  |  5.00 
  12    |   0.010770   | 99.711934  | 0.008695 | 99.81  |  4.98 
  13    |   0.010019   | 99.761317  | 0.009018 | 99.7

In [21]:
mu.saveModel(title+'no_gudu', no_model)

In [22]:
no_model = mu.getModel(title+'no_gudu')
print(no_model)

OCR_rnn(
  (emb): Embedding(121, 100)
  (lstm1): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(100, 200, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc1): Linear(in_features=800, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [23]:
loss, acc = tester.test(test_dataloader=no_test_dataloader,
                        device=device,
                        model=no_model,
                        title=title+'no_gudu')

test loss:  0.0138268363331008
test acc:  99.7
saved precision and recall results to file!


In [24]:
with open('../result/final_split', 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [25]:
mu.graphModel(no_train_dataloader, no_model, writer)

uploaded model graph to tensorboard!
